In [ ]:
import csv
import requests
import time
import os

# --- 1. 설정 ---
# 카카오 REST API 키 (제공해주신 키를 직접 포함했습니다)
KAKAO_REST_API_KEY = "a4b8bc5e20047154743b26622ce30e02"

# 입력 CSV 파일 경로
input_csv_file_path = "/home/momo/dev_ws/eda/data/Data_saramin.csv"
# 출력 CSV 파일 경로 (새로운 위도/경도 정보가 추가됨)
output_csv_file_path = "/home/momo/dev_ws/eda/data/Data_saramin_with_coordinates.csv"

# 입력 CSV 파일에서 주소 정보가 담긴 컬럼의 이름
address_column_name = "주소" # <-- CSV 파일의 실제 주소 컬럼명으로 변경해주세요!

# 카카오 로컬 API 주소-좌표 변환 엔드포인트
KAKAO_API_URL = "https://dapi.kakao.com/v2/local/search/address.json"

# --- 2. 카카오 API를 이용한 주소 -> 좌표 변환 함수 ---
def get_coordinates_from_address(address):
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"} # API 키가 포함된 헤더
    params = {"query": address}

    try:
        response = requests.get(KAKAO_API_URL, headers=headers, params=params)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생 (예: 401 Unauthorized, 400 Bad Request 등)
        data = response.json()

        if data and data.get('documents'):
            # 첫 번째 검색 결과 사용
            document = data['documents'][0]
            latitude = document.get('y') # 위도 (latitude)
            longitude = document.get('x') # 경도 (longitude)
            return latitude, longitude
        else:
            return None, None # 결과 없음 (예: 주소를 찾을 수 없는 경우)
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류 발생: {e}")
        # 오류 상세 정보 출력 (예: 401 Unauthorized 에러 시 키 문제)
        if response is not None:
            print(f"  HTTP 상태 코드: {response.status_code}")
            print(f"  API 응답: {response.text}")
        return None, None
    except Exception as e:
        print(f"좌표 변환 중 예상치 못한 오류 발생: {e}")
        return None, None

# --- 3. CSV 파일 읽기 및 좌표 추출 ---
all_data = []
processed_count = 0
failed_count = 0

print(f"'{input_csv_file_path}' 파일에서 주소를 읽고 좌표를 추출합니다...")

try:
    with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        
        # 새로운 CSV 파일의 헤더 정의 (기존 컬럼 + 위도, 경도)
        # 필드명이 이미 있으면 추가하지 않도록 확인 (이전 실행 중단 후 다시 시작 시 유용)
        fieldnames = reader.fieldnames
        if '위도' not in fieldnames:
            fieldnames.append('위도')
        if '경도' not in fieldnames:
            fieldnames.append('경도')

        for row in reader:
            address = row.get(address_column_name) # 주소 컬럼의 값 가져오기
            
            if address:
                latitude, longitude = get_coordinates_from_address(address)
                if latitude is not None and longitude is not None:
                    row['위도'] = latitude
                    row['경도'] = longitude
                    print(f"  처리됨 ({processed_count + 1}): '{address}' -> 위도: {latitude}, 경도: {longitude}")
                    processed_count += 1
                else:
                    row['위도'] = '좌표 추출 실패'
                    row['경도'] = '좌표 추출 실패'
                    print(f"  실패 ({failed_count + 1}): '{address}' -> 좌표 추출 실패 (API 결과 없음 또는 오류)")
                    failed_count += 1
            else:
                row['위도'] = '주소 없음'
                row['경도'] = '주소 없음'
                print(f"  경고: 주소 컬럼 '{address_column_name}'의 값이 비어있습니다. 행: {row}")
                failed_count += 1
            
            all_data.append(row)
            
            # 카카오 API의 호출 제한을 고려하여 요청 사이에 딜레이를 줍니다.
            time.sleep(0.1) # 0.1초 딜레이 (초당 최대 10회 요청)

except FileNotFoundError:
    print(f"오류: 입력 파일 '{input_csv_file_path}'을(를) 찾을 수 없습니다.")
    exit()
except KeyError:
    print(f"오류: CSV 파일에 '{address_column_name}' 컬럼이 없습니다. 컬럼명을 다시 확인해주세요.")
    exit()
except Exception as e:
    print(f"파일 처리 중 예상치 못한 오류가 발생했습니다: {e}")
    exit()

print(f"\n--- 좌표 추출 결과 ---")
print(f"총 {len(all_data)}개 주소 중:")
print(f"  성공적으로 처리된 주소: {processed_count}개")
print(f"  처리 실패/누락된 주소: {failed_count}개")

# --- 4. 결과를 새 CSV 파일로 저장 ---
print(f"결과를 '{output_csv_file_path}'에 저장합니다...")
try:
    # 출력 디렉토리가 없으면 생성
    output_dir = os.path.dirname(output_csv_file_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"출력 디렉토리 '{output_dir}' 생성 완료.")

    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader() # 헤더 쓰기
        writer.writerows(all_data) # 모든 데이터 쓰기
    print(f"데이터가 '{output_csv_file_path}'에 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"결과 CSV 파일 저장 중 오류가 발생했습니다: {e}")